In [2]:
import networkx as nx
import random
import numpy as np
from scipy.special import zeta
import math


In [3]:
def randomG(n, p):
    G = nx.Graph()

    for i in range(n):
        for j in range(i+1,n):
            r = random.random()
            if r<= p:
                G.add_edge(i,j)

    return G

In [4]:
def configurationG(deg):
    # deg is a list of numbers, the degree sequence of the nodes
    G = nx.Graph()
    nodes = [range(len(deg))]
    threshold = len(nodes)^2

    while len(nodes) > 1:
        done = False
        fail = False
        tries = 1

        while not done:
            edge = random.sample(nodes,2)
            if not G.has_edge(*edge):
                done = True
            
            else:
                tries += 1
                if tries > threshold:
                    fail = False
                    done = True
            
            if not fail:
                G.add_edge(*edge)
                deg[edge[0]] -= 1
                if deg[edge[0]] == 0:
                    nodes.remove(edge[0])
                    
                deg[edge[1]] -= 1
                if deg[edge[1]] == 0:
                    nodes.remove(edge[1])



    return G

s = configurationG([1,2,4,3])
s.edges()

EdgeView([])

In [5]:
# n number of nodes
# power is the exponent of the power law distribution
# returns a degree sequenc that follows the power law
def power_law_degree(n, power):
    deg_list = []
    deg = 1 # smallest degree we want to use

    z = zeta(power) # normalisation factor? something like that
    sumation = 0

    while len(deg_list) < n:
        p = (1 / (deg**power)) / z
        num = math.ceil(p*n)

        for i in range(num):
            if len(deg_list) == n: break
            deg_list.append(deg)
            sumation += deg
        deg += 1


    if sumation % 2 != 0:
        deg_list[-1] += 1



    return deg_list


s = power_law_degree(10,2)
s

[1, 1, 1, 1, 1, 1, 1, 2, 2, 3]

In [9]:
# n = number of nodes
# p = probability
def preferentialG(n,p):
    G = nx.Graph()
    nodes =[] 

    for u in range(n):
        r = random.random()
        if r < p and nodes != []:
            v = random.choice(nodes)
            G.add_edge(u,v)
            nodes.append(v)
            nodes.append(u)
        
        else:
            v = random.choice([x for x in range(n) if x != u])
            G.add_edge(u,v)
            nodes.append(v)
            nodes.append(u)


    return G

s = preferentialG(10,0.8)
s.edges()

EdgeView([(0, 4), (0, 8), (4, 1), (4, 2), (4, 5), (1, 2), (1, 3), (1, 6), (6, 7), (6, 9)])

In [18]:
# Watts - Strogatz graph
# n is number of nodes
# r is the radius of each node, edges within radius has strong ties.
# k is the number of random edges for each node, weak ties
# q is ruling the probability of having a weak tie within a distance
def genWS2DG(n, r,k,q):
    G = nx.Graph()
    nodes = {}
    prob = {}

    dim = math.sqrt(n)

    for i in range(n):
        x = random.random()
        y = random.random()
        nodes[i] = [x*dim, y*dim]
        prob[i] = {}

    for i in range(n):
        for j in range(i+1,n):
            dist = math.dist(nodes[i],nodes[j])
            prob[i][j] = 1/(dist**q)
            prob[j][i] = 1/(dist**q)
            
            if dist <= r:
                G.add_edge(i,j)

        norm = sum(prob[i].values())
        
        for h in range(k):
            s = np.random.choice([x for x in range(n) if x!= i], p = [prob[i][x]/norm for x in range(n) if x!= i] )
            G.add_edge(i,s)

    return G

s = genWS2DG(10,0.2,2,3)
s.edges()

EdgeView([(0, 4), (0, 6), (4, 1), (4, 3), (6, 5), (6, 8), (1, 3), (2, 7), (3, 9), (5, 9)])

In [29]:
# n is nubmer of nodes
# m is number of communities
# q is probability of edge within community and 1-q is edge between nodes in differnet communities
# c is the max number of communities a node can belong to
# s is the number of weak ties????
# p is ??
def affiliationG(n,m,q,c,p,s):
    G = nx.Graph()

    community = {}
    for i in range(m):
        community[i] = set()
    comm_inv = {}
    for i in range(n):
        comm_inv[i] = set()

    communities = [] # contains nodes, a nodes appear once for each community it belongs to
    nodes = []       # contains the degree???
    
    for i in range(n):
        r = random.random()
        if r <= q and len(communities) > 0:
            prot = random.choice(communities) # choose a random node

            for comm in comm_inv[prot]:
                community[comm].add(i)
            
                if comm not in comm_inv:
                    comm_inv[i].add(comm)
                    communities.append(i)
        else:
            num_com = random.randint(1,c)
            for k in range(num_com):
                comm = random.choice([x for x in range(m)])
                community[comm].add(i)
            
                if comm not in comm_inv:
                    comm_inv[i].add(comm)
                    communities.append(i)

        for comm in comm_inv[i]:
            for j in community[comm]:
                if j != i and not G.has_edge(i,j):
                    r2 = random.random()
                    if r2 <= p:
                        G.add_edge(i,j)
                        nodes.append(i)
                        nodes.append(j)

        for k in range(s):
            if len(nodes) == 0:
                v = random.choice([x for x in range(n) if x != i])
            else:
                v = random.choice(nodes)
            if v != i and G.has_edge(i,v):
                G.add_edge(i,v)
                nodes.append(i)
                nodes.append(v)


    return G
s = affiliationG(n = 10,m = 2,q = 0.9,c = 2,p = 0.4,s = 3)
s.edges()

EdgeView([])